### Data collection

In [329]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import random


import scipy
import math
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split 
from scipy.sparse.linalg import svds 
from sklearn import preprocessing 

import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lars\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [330]:
path='./Data/'

df_books=pd.read_csv(path +'filtered_books.csv') 
df_ratings=pd.read_csv(path + 'Ratings.csv') 
df_users=pd.read_csv(path + 'Users.csv') 

C:\Users\Lars\AppData\Local\Temp\ipykernel_25340\2624249305.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books=pd.read_csv(path +'filtered_books.csv')


In [331]:
df_ratings.shape

(1149780, 3)

In [332]:
df_users.describe()



,User-ID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


### Preprocessing

In [333]:
merged_df=pd.merge(df_users,df_ratings,on='User-ID')
merged_df=pd.merge(merged_df,df_books,on='ISBN')

merged_df.shape



(851511, 12)

In [334]:
book_review_counts = df_ratings['ISBN'].value_counts()
popular_books = book_review_counts[book_review_counts >= 20].index
filtered_ratings = df_ratings[df_ratings['ISBN'].isin(popular_books)]
#filtered_ratings.to_csv('data/filtered_ratings.csv', index=False)


### Collaberative Filtering

In [335]:
# Import necessary libraries
"""
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix

# Load the dataset
data = pd.read_csv('data/ratings.csv')
data.fillna(0, inplace=True)



book_review_counts = data['ISBN'].value_counts()
popular_books = book_review_counts[book_review_counts >= 20].index
data = data[data['ISBN'].isin(popular_books)]


user_rating_counts = data['User-ID'].value_counts()
active_users = user_rating_counts[user_rating_counts >= 5].index
data = data[data['User-ID'].isin(active_users)]



# Preprocess the data

# Create a user-item matrix
user_item_matrix = data.pivot(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)

# Convert the user-item matrix to a sparse matrix
user_item_matrix_sparse = csr_matrix(user_item_matrix.values).astype('float32')

# Normalize the user-item matrix
scaler = StandardScaler(with_mean=False)
user_item_matrix_normalized = scaler.fit_transform(user_item_matrix_sparse)


# Compute user-user similarity


user_similarity = cosine_similarity(user_item_matrix_normalized)


# Convert the similarity matrix back to a DataFrame
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)


# Function to get top N similar users
def get_top_n_similar_users(user_id, n):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).head(n+1).index.tolist()
    print("similar users: ",similar_users)
    similar_users.remove(user_id) # Remove the user_id itself from the list
    return similar_users

# Function to generate recommendations for a user
def recommend_items(user_id, n):
    similar_users = get_top_n_similar_users(user_id, n)
    similar_users_ratings = user_item_matrix.loc[similar_users]

    user_similarities = user_similarity_df.loc[user_id, similar_users]

    # Calculate weighted scores
    weighted_scores = similar_users_ratings.T.dot(user_similarities).div(user_similarities.sum())
    
    # Ignore items already rated by the target user
    target_user_ratings = user_item_matrix.loc[user_id]
    weighted_scores[target_user_ratings > 0] = 0
    
    # Return the top-N recommendations
    return weighted_scores.sort_values(ascending=False).head(n)



"""


'\nimport pandas as pd\nfrom sklearn.metrics.pairwise import cosine_similarity\nfrom sklearn.preprocessing import StandardScaler\nfrom scipy.sparse import csr_matrix\n\n# Load the dataset\ndata = pd.read_csv(\'data/ratings.csv\')\ndata.fillna(0, inplace=True)\n\n\n\nbook_review_counts = data[\'ISBN\'].value_counts()\npopular_books = book_review_counts[book_review_counts >= 20].index\ndata = data[data[\'ISBN\'].isin(popular_books)]\n\n\nuser_rating_counts = data[\'User-ID\'].value_counts()\nactive_users = user_rating_counts[user_rating_counts >= 5].index\ndata = data[data[\'User-ID\'].isin(active_users)]\n\n\n\n# Preprocess the data\n\n# Create a user-item matrix\nuser_item_matrix = data.pivot(index=\'User-ID\', columns=\'ISBN\', values=\'Book-Rating\').fillna(0)\n\n# Convert the user-item matrix to a sparse matrix\nuser_item_matrix_sparse = csr_matrix(user_item_matrix.values).astype(\'float32\')\n\n# Normalize the user-item matrix\nscaler = StandardScaler(with_mean=False)\nuser_item_ma

In [336]:
"""
# Example usage
#print(user_similarity_df.head)
user_id = 121138
recommendations = recommend_items(user_id, 20)
print(f"Recommendations for user {user_id}: {recommendations}")

#user_similarity_df.shape
#user_similarity_df.head()
"""

'\n# Example usage\n#print(user_similarity_df.head)\nuser_id = 121138\nrecommendations = recommend_items(user_id, 20)\nprint(f"Recommendations for user {user_id}: {recommendations}")\n\n#user_similarity_df.shape\n#user_similarity_df.head()\n'

In [337]:

"""
book_review_counts = df_ratings['ISBN'].value_counts()
popular_books = book_review_counts[book_review_counts >= 20].index
filtered_ratings = df_ratings[df_ratings['ISBN'].isin(popular_books)]

user_rating_counts = filtered_ratings['User-ID'].value_counts()
active_users = user_rating_counts[user_rating_counts >= 5].index
filtered_ratings = filtered_ratings[filtered_ratings['User-ID'].isin(active_users)]


user_item_matrix = filtered_ratings.pivot(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)



sparse_matrix = csr_matrix(user_item_matrix.values)
sparse_matrix=sparse_matrix.astype('float32')


item_similarity = cosine_similarity(sparse_matrix.T)  # Transpose for item-item similarity
item_similarity_df = pd.DataFrame(
    item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns
)

print("shape: ",item_similarity_df.shape)



# Step 4: Define a function to predict ratings
def predict_ratings(user_id, user_item_matrix, item_similarity_df, k=5):
    
    user_ratings = user_item_matrix.loc[user_id]
    
    # Align user's ratings with item_similarity_df index (ISBNs)
    user_ratings = user_ratings.reindex(item_similarity_df.index, fill_value=0)
    
    #print("User ratings", user_item_matrix.shape)
    
    # Compute weighted average of item similarities and user ratings
    scores = item_similarity_df.dot(user_ratings)
    
    
    scores[user_ratings > 0] = 0

    
    #print(scores)
    
    # Return top-k recommendations
    return scores.sort_values(ascending=False).head(k)



example_user_id = 165  # Choose the first user as an example
recommendations = predict_ratings(example_user_id, user_item_matrix, item_similarity_df, k=20)

recommendations

"""

'\nbook_review_counts = df_ratings[\'ISBN\'].value_counts()\npopular_books = book_review_counts[book_review_counts >= 20].index\nfiltered_ratings = df_ratings[df_ratings[\'ISBN\'].isin(popular_books)]\n\nuser_rating_counts = filtered_ratings[\'User-ID\'].value_counts()\nactive_users = user_rating_counts[user_rating_counts >= 5].index\nfiltered_ratings = filtered_ratings[filtered_ratings[\'User-ID\'].isin(active_users)]\n\n\nuser_item_matrix = filtered_ratings.pivot(index=\'User-ID\', columns=\'ISBN\', values=\'Book-Rating\').fillna(0)\n\n\n\nsparse_matrix = csr_matrix(user_item_matrix.values)\nsparse_matrix=sparse_matrix.astype(\'float32\')\n\n\nitem_similarity = cosine_similarity(sparse_matrix.T)  # Transpose for item-item similarity\nitem_similarity_df = pd.DataFrame(\n    item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns\n)\n\nprint("shape: ",item_similarity_df.shape)\n\n\n\n# Step 4: Define a function to predict ratings\ndef predict_ratings(user_id, 

In [338]:

# Import necessary libraries
import pandas as pd
from NLP_TFIDF import get_recommendations
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load datasets
df_books = pd.read_csv('merged_books.csv')
df_ratings = pd.read_csv('Data/Ratings.csv')

# Filter the ratings dataset to include only the books present in merged_books.csv
filtered_ratings = df_ratings[df_ratings['ISBN'].isin(df_books['ISBN'])]

# Save the filtered dataset to a new CSV file
filtered_ratings.to_csv('filtered_ratings.csv', index=False)

# Reload the books dataset
df_books = pd.read_csv('merged_books.csv')

# Compute TF-IDF matrix for content-based filtering
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_books['description'].fillna(''))

# Function to get the title of the recommended books
def get_title(book_id):
    if book_id in df_books['ISBN'].values:
        return df_books[df_books['ISBN'] == book_id]['Book-Title'].values[0]
    return None

# NLP-TFIDF Only Recommendation Function
def get_nlp_tfidf_recommendations(title, k=10):
    # Get content-based recommendations
    content_based_recommendations = get_recommendations(title)
    
    # Ensure we have at least k recommendations
    recommendations = content_based_recommendations[:k]
    
    return recommendations

# Function to get recommendations based on all books a user has read
def get_user_based_nlp_tfidf_recommendations(user_id, k=10):
    # Get the list of books the user has read
    user_books = filtered_ratings[filtered_ratings['User-ID'] == user_id]['ISBN'].unique()
    
    # Dictionary to store recommendations and their scores
    recommendation_scores = {}
    
    # Get recommendations for each book the user has read
    for book_id in user_books:
        book_title = get_title(book_id)
        if book_title:
            recommendations = get_nlp_tfidf_recommendations(book_title, k)
            for i, rec in enumerate(recommendations):
                score = k - i  # Assign scores from k to 1
                rec = df_books[df_books['Book-Title'] == rec]['ISBN'].values[0]

                if rec in recommendation_scores:
                    recommendation_scores[rec] += score
                else:
                    recommendation_scores[rec] = score
    
    print(recommendation_scores.items())
    # Sort recommendations by their scores
    sorted_recommendations = sorted(recommendation_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Ensure we have at least k recommendations
    final_recommendations = [rec for rec, score in sorted_recommendations[:k]]
 

    return final_recommendations

# Example usage
user_id = 165
book_title = "Destiny"
recommendations = get_user_based_nlp_tfidf_recommendations(user_id, k=10)
print(recommendations)
print(f"Content-Based Recommendations for user {user_id}:")
for rec in recommendations:
    print(rec)
    


dict_items([('0768322413', 10), ('0345362721', 9), ('0679408835', 8), ('0446517909', 7), ('0590257889', 6), ('0553802542', 5), ('0802132898', 4), ('0385491050', 3), ('0140042393', 2), ('015610685X', 1), ('0786003677', 10), ('0671870602', 9), ('0671685112', 8), ('0380792923', 7), ('1572460733', 6), ('0750925493', 5), ('0714530387', 4), ('0743467175', 3), ('1582430438', 2), ('0743418700', 1)])
['0768322413', '0786003677', '0345362721', '0671870602', '0679408835', '0671685112', '0446517909', '0380792923', '0590257889', '1572460733']
Content-Based Recommendations for user 165:
0768322413
0786003677
0345362721
0671870602
0679408835
0671685112
0446517909
0380792923
0590257889
1572460733


In [ ]:

from content_based_file import get_content_based_recommendations_by_user
from user_based_file import get_user_based_recommendations_by_user
from nlp_based_file import get_nlp_recommendations_by_user_program


#Variables for content based filtering

book_review_counts = df_ratings['ISBN'].value_counts()
popular_books = book_review_counts[book_review_counts >= 20].index
filtered_ratings = df_ratings[df_ratings['ISBN'].isin(popular_books)]

user_rating_counts = filtered_ratings['User-ID'].value_counts()
active_users = user_rating_counts[user_rating_counts >= 5].index
filtered_ratings = filtered_ratings[filtered_ratings['User-ID'].isin(active_users)]


user_item_matrix = filtered_ratings.pivot(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)



sparse_matrix = csr_matrix(user_item_matrix.values)
sparse_matrix=sparse_matrix.astype('float32')


item_similarity = cosine_similarity(sparse_matrix.T)  # Transpose for item-item similarity
item_similarity_df = pd.DataFrame(
    item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns
)



#print(testfunction)



example_user_id = 165  # Choose the first user as an example
recommendations = get_content_based_recommendations_by_user(example_user_id, user_item_matrix, item_similarity_df, 10)

print(recommendations)


ImportError: cannot import name 'get_nlp_recommendations_by_user_program' from 'nlp_based' (c:\Users\Lars\repos\school projects\H24\Book-recommender\nlp_based.py)

In [328]:
user_weight = 0.3
content_weight = 0.35
nlp_weight=0.35
top_n_recommendations = 50

example_user_id = 165



def scale_scores(scores, scale_to=10):
    max_score = scores.max()
    if max_score > 0:  # Avoid division by zero
        return (scores / max_score) * scale_to
    return scores



# User-based recommendations
def get_user_based_recommendations(user_id, n=top_n_recommendations):
    recommended_items = get_user_based_recommendations_by_user(user_id, n)
    return pd.DataFrame({
        'ISBN': recommended_items.index,
        'user_score': recommended_items.values
    })

# Content-based recommendations
def get_content_based_recommendations(user_id, n=top_n_recommendations):
    recommended_items = get_content_based_recommendations_by_user(user_id, user_item_matrix, item_similarity_df, k=n)
    return pd.DataFrame({
        'ISBN': recommended_items.index,
        'content_score': recommended_items.values
    })
    
    

    
def get_nlp_recommendations(user_id):
    n=books_read_by_user_with_description(user_id)*10
    recommended_items = get_nlp_recommendations_by_user_program(user_id, n)
    return pd.DataFrame({
        'ISBN': recommended_items,
        'nlp_score': range(n, 0, -1)  # Assign descending scores
    })
    

    
def books_read_by_user_with_description(user_id):
        user_books = filtered_ratings[filtered_ratings['User-ID'] == user_id]['ISBN'].unique()
        return len(user_books)

    
    
   

# Combine the three methods
def combine_recommendations(user_id, user_weight, content_weight,nlp_weight, n=top_n_recommendations):
    # Get top recommendations from both methods
    user_based_recs = get_user_based_recommendations(user_id, n)
    content_based_recs = get_content_based_recommendations(user_id, n)
    nlp_based_recs=get_nlp_recommendations(user_id)
    
     # Scale scores
    user_based_recs['user_score'] = scale_scores(user_based_recs['user_score'])
    content_based_recs['content_score'] = scale_scores(content_based_recs['content_score'])
    nlp_based_recs['nlp_score']=scale_scores(nlp_based_recs['nlp_score'])
    
    # Merge on ISBN
    combined = pd.merge(user_based_recs, content_based_recs, on='ISBN', how='outer')
    combined = pd.merge(combined, nlp_based_recs, on='ISBN', how='outer')

    
    # Fill missing scores with 0
    combined['user_score'] = combined['user_score'].fillna(0)
    combined['content_score'] = combined['content_score'].fillna(0)
    combined['nlp_score']=combined['nlp_score'].fillna(0)
    
    # Calculate hybrid score
    combined['hybrid_score'] = (user_weight * combined['user_score'] +
                                content_weight * combined['content_score']+
                                nlp_weight*combined['nlp_score'])
    
    # Sort by hybrid score
    combined = combined.sort_values(by='hybrid_score', ascending=False)
    
    # Return top recommendations
    return combined.head(40)

# Example usage


final_recommendations = combine_recommendations(example_user_id,user_weight , content_weight,nlp_weight ,n=top_n_recommendations)

print(final_recommendations)


similar users:  [165, 136733, 50711, 275610, 96354, 122235, 218836, 207727, 130793, 212009, 33036, 144348, 101299, 78545, 222941, 113618, 37790, 21870, 243607, 191913, 42759, 81854, 486, 28709, 257804, 112818, 96589, 216336, 248850, 112598, 194735, 66591, 250196, 93421, 110493, 189891, 6501, 147687, 82901, 65653, 231694, 179591, 242878, 110165, 80036, 143807, 136104, 266697, 46197, 185468, 55421]


NameError: name 'get_nlp_recommendations_by_user_program' is not defined